In [63]:
import pandas as pd
import numpy as np
import implicit
from scipy.sparse import csr_matrix

### Preparando os dados

In [64]:
df = pd.read_csv('../data/ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [65]:
movies_df = pd.read_csv('../data/ml-latest-small/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [66]:
def top_user_movies(userId, n=10):
    movies = df[df['userId'] == userId].sort_values('rating', ascending=False).head(n)
    return movies.merge(movies_df, on='movieId')[['title', 'rating']].reset_index(drop=True)

In [67]:
known_movies = movies_df['movieId'].unique()
# remove unknown movies
df = df[df['movieId'].isin(known_movies)]

In [68]:
df['userId'].nunique(), df['movieId'].nunique()

(610, 9724)

In [69]:
# m is a pivot of userId and movieId, rating is the value
data = df.pivot(index='userId', columns='movieId', values='rating')
data

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
m = np.array(data)
# replace nan with 0
m[np.isnan(m)] = 0
m = csr_matrix(m)
m

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100836 stored elements and shape (610, 9724)>

### Construindo o sistema com Alternating least squares

#### Montando o problema

Vamos otimizar nosso sistema de recomendação usando **Alternating least squares**.

Para entender a técnica, usei [essas notas de aula](https://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf) como referência,
que explica o método já em vista do problema de _user ratings_.

Suponhamos que temos uma lista com $n$ usuários e $m$ itens (no nosso caso, os itens são filmes).

Podemos construir uma matriz $R \in \mathbb{R}^{n \times m}$
onde cada elemento $r_{ui}$ é a nota dada pelo usuário $u$ para o item $i$.

Como vimos acima, é natural que essa matriz $R$ tenha muitos dados faltantes,
pois a maioria dos usuários não viu avaliou todos os filmes.

Desejamos preencher esses dados faltantes de forma que os valores encontrados sejam uma previsão da nota que o usuário $u$ daria para o filme $i$.

A abordagem escolhida envolve **decomposição matricial**.

Escolhemos um número de fatores $k$, relativamente pequeno (no exemplo, tomamos $k = 64$, o que é pequeno dado que temos 610 usuários e 9724 filmes).

Reduzimos, então, cada usuário $u$ em um vetor $x_u \in \mathbb{R}^k$, e cada item $i$ em um vetor $y_i \in \mathbb{R}^k$.
Esse vetores são chamados de **fatores**.

Então, para predizer a nota de um usuário $u$ para um filme $i$, basta predizermos $r_{ui} \approx x_u^Ty_i$.

Considerando que temos $n$ usuários e $m$ filmes, podemos escrever o problema em forma matricial
usando $X \in \mathbb{R}^{k \times n}$ e $Y \in \mathbb{R}^{k \times m},$
$$
X = \begin{bmatrix}
\vert & \ & \vert\\
x_1 & \cdots & x_n\\
\vert & \ & \vert
\end{bmatrix}, \quad
Y = \begin{bmatrix}
\vert & \ & \vert\\
y_1 & \cdots & y_m\\
\vert & \ & \vert
\end{bmatrix}.
$$

Nosso objetivo é, portanto, encontrar $X, Y$ tais que $R \approx X^TY$.

Escrevendo-o como um problema de mínimos quadrados, temos:

$$
\min_{X,Y} \sum_{r_{ui} \ \text{observados}} (r_{ui} - x_u^Ty_i)^2 + \lambda \left( \sum_u ||x_u||^2 + \sum_i ||y_i||^2 \right),
$$

onde $\lambda$ é um parâmetro de regularização.

Trata-se, portanto, de um problema **não convexo**, NP-hard de otimização.

Portanto, o natural seria resolvê-lo com alguma técnica como Gradientes Descendentes.

Contudo, o truque aqui é alternar entre otimizar $X$ e otimizar $Y$.
Ou seja, em um passo da nossa otimização trataremos $X$ como constante, e no outro trataremos $Y$ como constante, alternadamente,
até a convergência. Daí o nome da técnica, **Alernating least squares** (mínimos quadrados alternados).

Desenhamos, por fim, um algoritmo para o método

##### Alternating least squares

Com $X, Y$ aleatoriamente iniciados,

**repetir**

para $u = 1 \ldots n$
$$x_u = \left( \sum_{r_{ui} \in r_{u*}} y_i y_i^T + \lambda I_k \right)^{-1} \sum_{r_{ui} \in r_{u*}} r_{ui}y_i$$

para $i = 1 \ldots m$
$$y_i = \left( \sum_{r_{ui} \in r_{*i}} x_u x_u^T + \lambda I_k \right)^{-1} \sum_{r_{ui} \in r_{*i}} r_{ui}x_u$$

**até convergir**

A aula ainda apresenta algumas técnicas para implementação computacional do Alternating Least Squares.

Aqui, utilizei a versão trazida pelo pacote [implicit](https://github.com/benfred/implicit).

Vamos ajustar um modelo para nossa matriz já criada.

In [78]:
model = implicit.als.AlternatingLeastSquares(factors=64, random_state=42)

In [79]:
model.fit(m)

100%|██████████| 15/15 [00:00<00:00, 50.56it/s]


Peguemos uma lista de recomendações para o usuário $u = 0$.

In [80]:
recommended = model.recommend(0, m[0])
recommended

(array([1543,  793,  957, 1066, 2109,  921,  895, 1444, 1458,  506],
       dtype=int32),
 array([1.0386733 , 1.013742  , 0.92989504, 0.88418806, 0.8797215 ,
        0.851634  , 0.8476546 , 0.846146  , 0.8211517 , 0.813838  ],
       dtype=float32))

Esses são os filmes recomendados:

In [81]:
movies_df['title'][recommended[0]]

1543                              Jungle Book, The (1967)
793                                       Die Hard (1988)
957                                   Shining, The (1980)
1066                                   Under Siege (1992)
2109                            Pelican Brief, The (1993)
921                            Blues Brothers, The (1980)
895                               Paris Is Burning (1990)
1444                                     Labyrinth (1986)
1458    Friday the 13th Part VIII: Jason Takes Manhatt...
506                                        Aladdin (1992)
Name: title, dtype: object

Vamos ver os top 20 filmes favoritos do usuário...

In [82]:
# u = 0 => userId = 1
top_user_movies(1, 20)

,title,rating
0,Seven (a.k.a. Se7en) (1995),5.0
1,"Usual Suspects, The (1995)",5.0
2,Bottle Rocket (1996),5.0
3,Dumb & Dumber (Dumb and Dumber) (1994),5.0
4,Billy Madison (1995),5.0
5,Desperado (1995),5.0
6,Canadian Bacon (1995),5.0
7,Rob Roy (1995),5.0
8,Pinocchio (1940),5.0
9,Tombstone (1993),5.0


As recomendações fazem muito sentido!

Veja até que Mogli de 1967 foi recomendado,
e o usuário avaliou muito bem a versão de 1994.

In [83]:
# count how many movies with title "Jungle Book, The (1994)"
movies_df[movies_df['title'].str.contains('Jungle Book, The')]

,movieId,title,genres
320,362,"Jungle Book, The (1994)",Adventure|Children|Romance
1543,2078,"Jungle Book, The (1967)",Animation|Children|Comedy|Musical
